# 8 - Maxwell equations 

Topics covered in this tutorial:

- instance of [Maxwell](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.Maxwell) propagator
- initialization with noise
- power spectrum plot

## Light wave dispersion relation

In [ ]:
# set up domain Omega
from struphy.geometry.domains import Cuboid

l1 = 0.
r1 = 1.
l2 = 0.
r2 = 1.
l3 = 0.
r3 = 20.
domain = Cuboid(l1=l1, r1=r1, l2=l2, r2=r2, l3=l3, r3=r3)

In [ ]:
# set up Derham complex
from struphy.feec.psydac_derham import Derham

Nel = [1, 1, 128]
p = [1, 1, 3]
spl_kind = [True, True, True]
derham = Derham(Nel, p, spl_kind)

In [ ]:
# set up mass matrices
from struphy.feec.mass import WeightedMassOperators

mass_ops = WeightedMassOperators(derham, domain)

In [ ]:
# create solution field E in Vh_1 subset H(curl) 
e_field = derham.create_spline_function('electric field', 'Hcurl')

# create solution field B in Vh_2 subset H(div) 
b_field = derham.create_spline_function('magnetic field', 'Hdiv')

In [ ]:
# initial perturbations
pert_params_e = {"noise": {"comps": [True, True, False],
                           'direction' : 'e3',
                           'amp' : 0.1, 
                           'seed' : None,}}

pert_params_b = {"noise": {"comps": [False, False, False],
                           'direction' : 'e3',
                           'amp' : 0.1, 
                           'seed' : None,}}

In [ ]:
e_field.initialize_coeffs(pert_params=pert_params_e)
b_field.initialize_coeffs(pert_params=pert_params_b)

In [ ]:
# evalaute at logical coordinates
import numpy as np

e1 = .5
e2 = .5
e3 = np.linspace(0, 1, 100)

e_vals = e_field(e1, e2, e3, squeeze_out=True)
b_vals = b_field(e1, e2, e3, squeeze_out=True)

In [ ]:
e_vals

In [ ]:
from struphy.propagators.propagators_fields import Maxwell

# default parameters of the Propagator
opts = Maxwell.options(default=True)
opts

In [ ]:
# pass simulation parameters to Propagator
Maxwell.derham = derham
Maxwell.domain = domain
Maxwell.mass_ops = mass_ops

In [ ]:
prop_implicit = Maxwell(e_field.vector, b_field.vector)
prop_rk4 = Maxwell(e_field.vector, b_field.vector, algo='rk4')

In [ ]:
Tend = 100. - 1e-6
dt = .05

In [ ]:
# implicit time stepping
Ex_of_t_implicit = {}
time = 0.
n = 0
while time < Tend:
    n += 1
    
    # advance in time
    prop_implicit(dt)
    time += dt
    
    # evaluate solution and push to Omega
    Ex_of_t_implicit[time] = e_field(e1, e2, e3)
    
    if n % 100 == 0:
        print(f'{n}/{int(np.ceil(Tend/dt))} steps completed with {prop_implicit._algo =}.')

In [ ]:
# reset initial condition
e_field.initialize_coeffs(pert_params=pert_params_e)
b_field.initialize_coeffs(pert_params=pert_params_b)

In [ ]:
# rk4 time stepping
Ex_of_t_rk4 = {}
time = 0.
n = 0
while time < Tend:
    n += 1
    
    # advance in time
    prop_rk4(dt)
    time += dt
    
    # evaluate solution and push to Omega
    Ex_of_t_rk4[time] = e_field(e1, e2, e3)
    
    if n % 100 == 0:
        print(f'{n}/{int(np.ceil(Tend/dt))} steps completed with {prop_rk4._algo = }.')

In [ ]:
from struphy.diagnostics.diagn_tools import power_spectrum_2d
x, y, z = domain(e1, e2, e3)

# fft in (t, z) of first component of e_field on physical grid
power_spectrum_2d(Ex_of_t_implicit,
                  'e1',
                  'Maxwell',
                  grids=[e1, e2, e3],
                  grids_mapped=[x, y, z],
                  component=0,
                  slice_at=[0, 0, None],
                  do_plot=True,
                  disp_name='Maxwell1D')

In [ ]:
# fft in (t, z) of first component of e_field on physical grid
power_spectrum_2d(Ex_of_t_rk4,
                  'e1',
                  'Maxwell',
                  grids=[e1, e2, e3],
                  grids_mapped=[x, y, z],
                  component=0,
                  slice_at=[0, 0, None],
                  do_plot=True,
                  disp_name='Maxwell1D')

## Cylindrical wave guide

Under construction ...